# Experimenting Around

## First Trades

In [2]:
import os
import asyncio
import threading
from multiprocessing import Process
import numpy as np
from alpaca.trading.client import TradingClient
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.stream import TradingStream
import nest_asyncio # used for running event loops within Jupyter's event loop
import math

In [3]:
nest_asyncio.apply() # allows event loops to be run within jupyter

api_key = os.getenv("Public_Key")
secret_key = os.getenv("Secret")
client = TradingClient(api_key,secret_key,paper=True)
account = dict(client.get_account())
for k,v in account.items():
    print(f"{k:30}{v}")



id                            54404f7b-795a-4436-a230-7eb13b70da1b
account_number                PA3A0K6YRNTJ
status                        AccountStatus.ACTIVE
crypto_status                 AccountStatus.ACTIVE
currency                      USD
buying_power                  124338.7
regt_buying_power             124338.7
daytrading_buying_power       0
non_marginable_buying_power   41536.9
cash                          24461.99
accrued_fees                  0
pending_transfer_out          None
pending_transfer_in           None
portfolio_value               99876.71
pattern_day_trader            False
trading_blocked               False
transfers_blocked             False
account_blocked               False
created_at                    2024-11-29 16:10:55.429486+00:00
trade_suspended_by_user       False
multiplier                    2
shorting_enabled              True
equity                        99876.71
last_equity                   99340.39
long_market_value             75414.72

In [10]:
order_data = MarketOrderRequest(
    symbol = "AMD",
    qty = 100,
    side = OrderSide.SELL,
    time_in_force = TimeInForce.DAY
)
    
order = client.submit_order(order_data = order_data) # places the order
print(order)
trades = TradingStream(api_key,secret_key,paper=True)
async def trade_status(data):
    print(data)
async def main():
    trades.subscribe_trade_updates(trade_status)
    trades.run()  # Await to handle the asynchronous nature properly
def x():
    asyncio.run(main())
thread = Process(target = main)

# Run the main function
thread.start()
thread.join()
# await main()


id=UUID('80bd6124-987c-4b5c-afb5-ba19d55817ad') client_order_id='c97a1c10-fcda-4a7b-b2b8-d4cb41d14f7a' created_at=datetime.datetime(2024, 12, 24, 15, 40, 14, 151263, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2024, 12, 24, 15, 40, 14, 151669, tzinfo=TzInfo(UTC)) submitted_at=datetime.datetime(2024, 12, 24, 15, 40, 14, 151263, tzinfo=TzInfo(UTC)) filled_at=None expired_at=None canceled_at=None failed_at=None replaced_at=None replaced_by=None replaces=None asset_id=UUID('03fb07bb-5db1-4077-8dea-5d711b272625') symbol='AMD' asset_class=<AssetClass.US_EQUITY: 'us_equity'> notional=None qty='100' filled_qty='0' filled_avg_price=None order_class=<OrderClass.SIMPLE: 'simple'> order_type=<OrderType.MARKET: 'market'> type=<OrderType.MARKET: 'market'> side=<OrderSide.SELL: 'sell'> time_in_force=<TimeInForce.DAY: 'day'> limit_price=None stop_price=None status=<OrderStatus.PENDING_NEW: 'pending_new'> extended_hours=False legs=None trail_percent=None trail_price=None hwm=None position_intent=<

## Trying Some Actual Strategies/Visualizations

In [5]:
!where python  


c:\Users\kaili\Desktop\Coding Projects\trading-bot\venv\Scripts\python.exe
C:\Users\kaili\AppData\Local\Programs\Python\Python312\python.exe


In [6]:
import quantstats as qs
import yfinance

stock = "MCD"
portfolio = qs.utils.download_returns(stock,period='5y')
print([f for f in dir(qs.stats) if f[0]!="_"])

['adjusted_sortino', 'autocorr_penalty', 'avg_loss', 'avg_return', 'avg_win', 'best', 'cagr', 'calmar', 'common_sense_ratio', 'comp', 'compare', 'compsum', 'conditional_value_at_risk', 'consecutive_losses', 'consecutive_wins', 'cpc_index', 'cvar', 'distribution', 'drawdown_details', 'expected_return', 'expected_shortfall', 'exposure', 'gain_to_pain_ratio', 'geometric_mean', 'ghpr', 'greeks', 'implied_volatility', 'information_ratio', 'kelly_criterion', 'kurtosis', 'max_drawdown', 'monthly_returns', 'omega', 'outlier_loss_ratio', 'outlier_win_ratio', 'outliers', 'payoff_ratio', 'pct_rank', 'probabilistic_adjusted_sortino_ratio', 'probabilistic_ratio', 'probabilistic_sharpe_ratio', 'probabilistic_sortino_ratio', 'profit_factor', 'profit_ratio', 'r2', 'r_squared', 'rar', 'recovery_factor', 'remove_outliers', 'risk_of_ruin', 'risk_return_ratio', 'rolling_greeks', 'rolling_sharpe', 'rolling_sortino', 'rolling_volatility', 'ror', 'serenity_index', 'sharpe', 'skew', 'smart_sharpe', 'smart_sor